# Data Analysis Grades of Portuguese Students
In this file the provided data will be analysed. For this the data are first loaded, then the specific value counts and counts of not known values researched. Finally a correlation matrix will be generated to detect correlations between different columns. On this basis the Bayesian network can be generated and later improved.

## 0. Install required libraries
This section installs all needed libraries in a python virtual environment which can be used as Jupyter kernel. This needs only to be done once.

In [20]:
%%python -m venv ../.env
%%source ../.env/bin/activate
%pip install -r ../requirements.txt

## 1. Import libraries
This section imports all needed libraries and sets the path to the data, number of rows to be displayed.

In [21]:
%reset -f

import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

# Set matplotlib options
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Set pandas options
pd.set_option('display.max_columns', 500)

# Constants
DATA_PATH = "../data/"
DOC_PATH = "../doc/"

## 2. Load data
Firstly, the data needs to be loaded. This is done by using the pandas library, with the read_csv function. The data is then displayed and stored in a dataframe.

In [22]:
original_data = pd.read_csv(DATA_PATH + "student-por_2.csv", sep=";")
original_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,Unnamed: 33
0,MS,M,16,R,GT3,T,1,1,at_home,other,other,father,2,1,0,no,no,no,yes,yes,yes,no,no,3,4,4,3,4,5,6,11,11,11,"0,581558765"
1,MS,F,18,R,GT3,T,2,2,other,other,other,mother,2,1,1,no,no,no,no,yes,no,yes,yes,5,5,5,1,1,3,0,8,6,0,"0,677773943"
2,MS,M,17,R,GT3,T,1,1,other,services,course,mother,2,1,0,no,yes,no,yes,no,yes,yes,yes,4,5,5,1,3,2,0,10,9,10,"0,58860641"
3,GP,M,18,U,LE3,T,1,1,other,other,course,mother,1,1,2,no,no,no,no,yes,no,yes,yes,2,3,5,2,5,4,0,11,9,0,"0,627079796"
4,GP,F,18,U,GT3,T,2,1,other,other,home,mother,1,2,0,no,yes,no,no,yes,yes,yes,yes,4,2,5,1,2,1,8,14,14,15,"0,459968853"


As can be seen, the data contains not only numerical values, but also categorical values. These categorical values need to be encoded, so that they can be used in the Bayesian network. But first, the data needs to be analysed more in detail. Therefore, the information about the data with columns, non-null values and data types is displayed. As a reminder, the data contains out of 33 columns, with 599 entries. The columns are described in the following table.

In [23]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   school       599 non-null    object
 1   sex          599 non-null    object
 2   age          599 non-null    int64 
 3   address      599 non-null    object
 4   famsize      599 non-null    object
 5   Pstatus      599 non-null    object
 6   Medu         599 non-null    int64 
 7   Fedu         599 non-null    int64 
 8   Mjob         599 non-null    object
 9   Fjob         599 non-null    object
 10  reason       599 non-null    object
 11  guardian     599 non-null    object
 12  traveltime   599 non-null    int64 
 13  studytime    599 non-null    int64 
 14  failures     599 non-null    int64 
 15  schoolsup    599 non-null    object
 16  famsup       599 non-null    object
 17  paid         599 non-null    object
 18  activities   599 non-null    object
 19  nursery      599 non-null    

Out of the table follows, that there are no not known values, so the data is already clean. But the data types are not perfect because the data contains objects, which are not numerical values. This needs to be changed for the columns: 

- school
- sex
- address
- famsize
- Pstatus
- Mjob
- Fjob
- reason
- guardian
- schoolsup
- famsup
- paid
- activities
- nursery
- higher
- internet
- romantic

Additionally, the column *Unnamed: 33* contains no information about the students and is not part of the data description. Therefore, this column will be dropped.

In [24]:
del original_data['Unnamed: 33']

Furthermore, some of the above columns contain only two values, which can be encoded as binary values. After a closer look into the data description, the following columns are encoded as binary values:

- school
- sex
- address
- Psatus
- schoolsup
- famsup
- paid
- activities
- nursery
- higher
- internet
- romantic

The other columns are encoded as categorical values. The encoding is done by using sklearn's OneHotEncoder. The encoder creates a new column for each value of the categorical column. The value of the new column is 1, if the value of the categorical column is the same as the value of the new column. Otherwise the value is 0. The new columns are added to the dataframe and the categorical columns are dropped. As a result, the dataframe contains only numerical values.

In [25]:
clean_data = original_data.copy()

# Binary columns to be converted to 0/1
clean_data['school'] = clean_data['school'].map({'GP': 0, 'MS': 1})
clean_data['sex'] = clean_data['sex'].map({'F': 0, 'M': 1})
clean_data['address'] = clean_data['address'].map({'U': 0, 'R': 1})
clean_data['famsize'] = clean_data['famsize'].map({'LE3': 0, 'GT3': 1})
clean_data['Pstatus'] = clean_data['Pstatus'].map({'T': 0, 'A': 1})
clean_data['schoolsup'] = clean_data['schoolsup'].map({'yes': 1, 'no': 0})
clean_data['famsup'] = clean_data['famsup'].map({'yes': 1, 'no': 0})
clean_data['paid'] = clean_data['paid'].map({'yes': 1, 'no': 0})
clean_data['activities'] = clean_data['activities'].map({'yes': 1, 'no': 0})
clean_data['nursery'] = clean_data['nursery'].map({'yes': 1, 'no': 0})
clean_data['higher'] = clean_data['higher'].map({'yes': 1, 'no': 0})
clean_data['internet'] = clean_data['internet'].map({'yes': 1, 'no': 0})
clean_data['romantic'] = clean_data['romantic'].map({'yes': 1, 'no': 0})

# Categories to be converted to one-hot encoding
converter = make_column_transformer((OneHotEncoder(), [
    # 'school',
    # 'sex',
    # 'address',
    'Mjob',
    'Fjob',
    'reason',
    'guardian',
    'G3'
    # 'traveltime',
    # 'studytime',
]), remainder='passthrough', verbose_feature_names_out=False)

# Create the new columns
numerical_data = converter.fit_transform(clean_data)
numerical_data = pd.DataFrame(numerical_data, columns=converter.get_feature_names_out())

# Delete the columns that are not useful
numerical_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 62 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Mjob_at_home       599 non-null    float64
 1   Mjob_health        599 non-null    float64
 2   Mjob_other         599 non-null    float64
 3   Mjob_services      599 non-null    float64
 4   Mjob_teacher       599 non-null    float64
 5   Fjob_at_home       599 non-null    float64
 6   Fjob_health        599 non-null    float64
 7   Fjob_other         599 non-null    float64
 8   Fjob_services      599 non-null    float64
 9   Fjob_teacher       599 non-null    float64
 10  reason_course      599 non-null    float64
 11  reason_home        599 non-null    float64
 12  reason_other       599 non-null    float64
 13  reason_reputation  599 non-null    float64
 14  guardian_father    599 non-null    float64
 15  guardian_mother    599 non-null    float64
 16  guardian_other     599 non

### 2.1. Correlation matrix

The data is now ready for the correlation matrix. For this, the pandas function corr is used. The result is displayed as a heatmap in the colors red and green. The red color indicates a negative correlation, the green color a positive correlation. The darker the color, the stronger the correlation. The correlation matrix is displayed in the following section and can be seen in the files *correlation-matrix.png* and *correlation-matrix.xlsx* in the data folder.

In [27]:
corr_styled = numerical_data.corr().style.background_gradient(cmap='RdYlGn', axis=None).set_precision(2)
# dfi.export(corr_styled, DOC_PATH + "correlation_matrix.png", max_cols=-1)
# corr_styled.to_excel(DOC_PATH + "correlation_matrix.xlsx")
corr_styled

/tmp/ipykernel_6733/841656423.py:1: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr_styled = numerical_data.corr().style.background_gradient(cmap='RdYlGn', axis=None).set_precision(2)


,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_at_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,G3_0,G3_1,G3_5,G3_6,G3_7,G3_8,G3_9,G3_10,G3_11,G3_12,G3_13,G3_14,G3_15,G3_16,G3_17,G3_18,G3_19,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
Mjob_at_home,1.00,-0.15,-0.41,-0.27,-0.19,0.14,-0.07,0.01,-0.01,-0.09,0.13,-0.08,0.07,-0.13,-0.01,-0.01,0.04,0.02,-0.02,-0.02,-0.03,0.00,0.02,0.12,0.08,0.13,-0.09,-0.06,-0.06,-0.05,-0.11,-0.05,0.07,-0.03,0.22,-0.14,0.08,0.17,-0.02,-0.04,-0.38,-0.21,0.19,-0.05,0.11,-0.02,-0.01,-0.01,-0.07,-0.01,-0.15,-0.24,0.04,-0.02,-0.02,-0.04,-0.02,-0.03,-0.09,-0.03,-0.17,-0.15
Mjob_health,-0.15,1.00,-0.22,-0.15,-0.10,-0.05,0.23,-0.08,0.02,-0.01,-0.11,-0.01,0.02,0.13,0.04,-0.01,-0.05,-0.05,-0.01,-0.01,-0.02,-0.03,0.02,-0.04,-0.03,-0.06,-0.03,-0.01,0.03,0.06,0.04,0.07,0.10,-0.02,-0.09,0.02,-0.09,-0.09,-0.01,-0.03,0.26,0.14,-0.13,-0.00,-0.05,-0.06,0.05,0.01,0.00,0.02,0.10,0.10,0.03,-0.04,-0.04,0.04,-0.09,0.01,0.01,-0.09,0.13,0.13
Mjob_other,-0.41,-0.22,1.00,-0.41,-0.29,-0.04,-0.11,0.24,-0.15,-0.10,-0.03,0.07,-0.05,0.01,0.07,-0.12,0.10,0.05,0.05,-0.03,0.01,-0.01,0.02,-0.00,0.02,-0.04,0.04,0.06,-0.03,-0.03,0.04,-0.09,-0.04,-0.05,0.03,-0.02,0.05,0.05,0.07,0.02,-0.25,-0.21,0.03,-0.01,0.01,0.06,-0.11,-0.03,-0.07,-0.09,-0.03,-0.07,0.03,0.01,-0.00,0.01,-0.01,-0.04,-0.01,0.04,-0.05,-0.06
Mjob_services,-0.27,-0.15,-0.41,1.00,-0.19,-0.02,-0.01,-0.13,0.16,0.00,-0.04,-0.02,0.00,0.07,-0.02,0.07,-0.09,-0.06,-0.02,0.08,0.04,0.07,0.00,-0.05,-0.02,-0.04,0.04,-0.00,0.00,0.06,0.01,0.03,-0.07,0.04,-0.12,0.07,-0.04,-0.10,-0.02,0.05,0.15,0.12,-0.07,0.03,0.02,0.03,0.07,0.05,0.08,0.04,0.04,0.14,-0.07,0.04,0.00,0.05,0.06,0.04,0.06,0.07,0.05,0.03
Mjob_teacher,-0.19,-0.10,-0.29,-0.19,1.00,-0.05,0.08,-0.14,0.02,0.27,0.02,0.04,-0.03,-0.04,-0.10,0.12,-0.05,0.01,-0.01,-0.01,-0.02,-0.04,-0.06,-0.06,-0.09,-0.02,0.04,0.01,0.09,-0.01,0.04,0.10,-0.01,0.07,-0.11,0.11,-0.06,-0.08,-0.05,-0.02,0.46,0.33,-0.10,0.04,-0.13,-0.06,0.04,-0.00,0.10,0.10,0.11,0.15,-0.03,-0.00,0.06,-0.06,0.04,0.04,0.04,-0.05,0.14,0.13
Fjob_at_home,0.14,-0.05,-0.04,-0.02,-0.05,1.00,-0.05,-0.31,-0.17,-0.06,0.11,-0.05,-0.03,-0.06,0.01,-0.03,0.04,-0.00,-0.01,-0.01,-0.02,0.08,0.02,-0.03,0.08,0.01,-0.05,-0.04,0.02,0.02,-0.07,-0.06,0.06,-0.02,0.15,-0.04,0.05,-0.02,-0.04,0.00,-0.10,-0.10,-0.03,-0.01,0.03,0.02,0.05,-0.01,0.04,0.03,-0.10,-0.12,0.01,-0.08,0.05,0.01,-0.03,-0.05,-0.05,0.03,-0.10,-0.08
Fjob_health,-0.07,0.23,-0.11,-0.01,0.08,-0.05,1.00,-0.21,-0.12,-0.04,-0.05,-0.06,0.02,0.10,0.07,-0.04,-0.05,-0.03,-0.01,-0.01,-0.01,-0.02,-0.00,-0.04,0.03,-0.08,0.06,0.01,-0.00,-0.05,0.07,0.05,0.04,-0.01,-0.08,-0.04,-0.11,-0.09,-0.01,-0.01,0.15,0.23,-0.12,0.09,-0.06,0.09,0.11,-0.01,0.00,0.07,0.06,0.04,-0.02,0.00,-0.07,-0.01,-0.05,-0.04,0.08,-0.02,0.08,0.08
Fjob_other,0.01,-0.08,0.24,-0.13,-0.14,-0.31,-0.21,1.00,-0.71,-0.27,-0.02,0.07,-0.08,0.01,-0.14,0.07,0.10,-0.01,-0.05,-0.05,-0.01,0.03,-0.10,0.08,-0.05,0.08,0.02,0.04,-0.03,0.06,-0.05,-0.08,-0.02,-0.01,-0.06,-0.02,0.05,0.05,-0.02,0.09,-0.13,-0.22,0.10,-0.03,0.04,-0.01,-0.06,0.01,-0.08,-0.05,-0.02,-0.00,0.01,0.02,0.03,0.03,-0.03,-0.01,0.08,0.01,-0.03,-0.01
Fjob_services,-0.01,0.02,-0.15,0.16,0.02,-0.17,-0.12,-0.71,1.00,-0.15,-0.01,-0.03,0.08,-0.02,0.10,-0.05,-0.08,0.02,0.07,0.07,0.03,-0.05,0.11,-0.05,0.05,-0.05,-0.00,-0.03,-0.01,-0.06,0.01,0.05,-0.01,-0.04,0.06,0.03,-0.02,0.00,0.00,-0.11,0.01,0.03,-0.02,0.01,0.00,-0.06,-0.02,0.01,0.04,0.01,0.03,0.05,-0.02,0.06,-0.05,-0.02,0.08,0.09,-0.10,-0.02,-0.02,-0.04
Fjob_teacher,-0.09,-0.01,-0.10,0.00,0.27,-0.06,-0.04,-0.27,-0.15,1.00,-0.02,-0.00,0.03,-0.00,0.03,0.01,-0.06,0.01,-0.01,-0.01,-0.01,-0.03,-0.02,-0.02,-0.10,-0.02,-0.03,-0.00,0.07,0.01,0.10,0.10,-0.03,0.11,-0.10,0.06,-0.06,-0.03,0.09,0.03,0.26,0.35,-0.05,-0.

### 2.2. Analysis of the Correlation matrix
TODO: Needs to be done